In [1]:
from sklearn.neural_network import MLPClassifier

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [3]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import scipy
from scipy import ndimage

In [4]:
# read data
X_train = pd.read_csv('input/trainData.csv')
y_train = pd.read_csv('input/trainLabels.csv')
X_test = pd.read_csv('input/kaggleTestSubset.csv')
y_test = pd.read_csv('input/kaggleTestSubsetLabels.csv')

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [6]:
models = []
models.append(MLPClassifier(activation="relu", solver = "adam", alpha=1e-5, hidden_layer_sizes=(700, 500, 250, 100), random_state=1, learning_rate="invscaling", warm_start=True))
models.append(MLPClassifier(activation="tanh", solver = "adam", alpha=1e-5, hidden_layer_sizes=(700, 500, 250, 100), random_state=1, learning_rate="invscaling", warm_start=True))
models.append(MLPClassifier(activation="logistic", solver = "adam", alpha=1e-5, hidden_layer_sizes=(700, 500, 250, 100), random_state=1, learning_rate="invscaling", warm_start=True))
models.append(MLPClassifier(activation="relu", solver = "sgd", alpha=1e-5, hidden_layer_sizes=(700, 500, 250, 100), random_state=1, learning_rate="invscaling", warm_start=True))
models.append(MLPClassifier(activation="tanh", solver = "sgd", alpha=1e-5, hidden_layer_sizes=(700, 500, 250, 100), random_state=1, learning_rate="invscaling", warm_start=True))
models.append(MLPClassifier(activation="logistic", solver = "sgd", alpha=1e-5, hidden_layer_sizes=(700, 500, 250, 100), random_state=1, learning_rate="invscaling", warm_start=True))

In [7]:
def test_model(m):
    fit_time = time.time()
    m.fit(X_train, y_train.Label)
    fit_time = time.time() - fit_time

    pred_time = time.time()
    pred = m.predict(X_test)
    pred_time = time.time() - pred_time
    
    acc = accuracy_score(y_test, pred)
    print("Accuracy: " + str(acc) + " Time to train: " + str(fit_time) + " Time to predict:" + str(pred_time))
    print ("=====================")

In [ ]:
for model in models:
    test_model(model)

Accuracy: 0.985242718447 Time to train: 39.448906898498535 Time to predict:0.14255380630493164
Accuracy: 0.985242718447 Time to train: 52.28204798698425 Time to predict:0.1663961410522461


In [45]:
model.activation

'relu'

In [78]:
from scipy.ndimage import interpolation

def moments(image):
    c0,c1 = np.mgrid[:image.shape[0],:image.shape[1]] # A trick in numPy to create a mesh grid
    totalImage = np.sum(image) #sum of pixels
    m0 = np.sum(c0*image)/totalImage #mu_x
    m1 = np.sum(c1*image)/totalImage #mu_y
    m00 = np.sum((c0-m0)**2*image)/totalImage #var(x)
    m11 = np.sum((c1-m1)**2*image)/totalImage #var(y)
    m01 = np.sum((c0-m0)*(c1-m1)*image)/totalImage #covariance(x,y)
    mu_vector = np.array([m0,m1]) # Notice that these are \mu_x, \mu_y respectively
    covariance_matrix = np.array([[m00,m01],[m01,m11]]) # Do you see a similarity between the covariance matrix
    return mu_vector, covariance_matrix

def deskew(image):
    c,v = moments(image)
    alpha = v[0,1]/v[0,0]
    affine = np.array([[1,0],[alpha,1]])
    ocenter = np.array(image.shape)/2.0
    offset = c-np.dot(affine,ocenter)
    return interpolation.affine_transform(image,affine,offset=offset)

def preprocess_X(X):
    result = []
    values = X.values
    for v in values:
        curr = v.reshape((28, 28))
        curr = deskew(curr)
        result.append(curr.flatten())
        
    return result


deskewed_train = preprocess_X(X_train/255)
deskewed_test = preprocess_X(X_test/255)

In [79]:
model.fit(deskewed_train, y_train.Label)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(700, 500, 250, 100), learning_rate='invscaling',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=True)

In [80]:
accuracy_score(y_test, model.predict(deskewed_test))

0.99300970873786409

In [83]:
# read data
X_test_full = pd.read_csv('input/testData.csv')
deskewed_test = preprocess_X(X_test_full/255)
predictions = model.predict(deskewed_test)

In [84]:
import csv
with open('submission1.csv', 'w') as file:
    fieldnames = ['ID', 'Label']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    
    writer.writeheader()
    i = 1
    for prediction in predictions:
        writer.writerow({'ID':i, 'Label':prediction})
        i += 1

In [12]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
labels = encoder.fit_transform(y_train)
labels_test = encoder.fit_transform(y_test)

In [61]:
y = model.predict(X_test)

# Convolutional Neural Network

In [34]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam

In [35]:
model = Sequential()

# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(100, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(50, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(0.25))

# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(600, activation='relu'))
model.add(Dense(400, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
adam = Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [36]:
X_train_im = X_train.values.reshape(len(X_train.values), 28, 28, 1)
X_test_im = X_test.values.reshape(len(X_test.values), 28, 28, 1)

In [37]:
model.fit(X_train_im, labels, batch_size=100, epochs=10)

Epoch 1/10
12019/12019 [==============================] - 80s 7ms/step - loss: 6.6509 - acc: 0.5769
Epoch 2/10
12019/12019 [==============================] - 84s 7ms/step - loss: 1.9929 - acc: 0.8623
Epoch 3/10
12019/12019 [==============================] - 84s 7ms/step - loss: 0.1190 - acc: 0.9814
Epoch 4/10
12019/12019 [==============================] - 81s 7ms/step - loss: 0.0410 - acc: 0.9897
Epoch 5/10
12019/12019 [==============================] - 78s 7ms/step - loss: 0.0226 - acc: 0.9929
Epoch 6/10
12019/12019 [==============================] - 79s 7ms/step - loss: 0.0173 - acc: 0.9951
Epoch 7/10
12019/12019 [==============================] - 79s 7ms/step - loss: 0.0126 - acc: 0.9964
Epoch 8/10
12019/12019 [==============================] - 79s 7ms/step - loss: 0.0138 - acc: 0.9953
Epoch 9/10
12019/12019 [==============================] - 78s 7ms/step - loss: 0.0108 - acc: 0.9969
Epoch 10/10
12019/12019 [==============================] - 78s 7ms/step - loss: 0.0069 - acc: 0.9979

In [38]:
model.evaluate(X_test_im, labels_test)

2575/2575 [==============================] - 6s 2ms/step


[0.019671346874156635, 0.99533980582524273]